In [1]:
%load_ext autoreload
%autoreload 2

In [79]:
# Import Module
from bs4 import BeautifulSoup
import os
import re
import pickle
import pandas as pd
import numpy as np
import collections

from utils import *

def extract_benchmark(text):
    words = text.split()
    # indices = np.where(words in ('benchmark', 'Benchmark'))[0]
    # indices = np.where(words == 'benchmark')[0]
    indices = [i for i, word in enumerate(words) if word.lower() in ("benchmark", "benchmarks")]
    potential_benchmarks = set()
    for i in indices:
        window = 15
        window_words = words[i - window:i + window]
        # print()
        for w in re.findall(r'“(.*?)”', ' '.join(window_words)) + re.findall(r'"(.*?)"', ' '.join(window_words)):
            potential_benchmarks.add(w)

        potential_words = list(filter(lambda w:
                                      any(c.isupper() for c in w),
                                      window_words))

        for i in range(1, 6):
            for j in range(len(potential_words) - i):
                pw = ' '.join(potential_words[j:j + i])  #todo check if indices are right
                potential_benchmarks.add(pw)

    return potential_benchmarks

In [104]:
df = df_from_filings()
# df = df.head(100)
df.shape

(2644, 5)

#### add potential benchmarks

In [105]:
potential_benchmarks_raw = df.apply(lambda row: extract_benchmark(row['text']), axis=1)
potential_benchmarks_all = pd.concat([potential_benchmarks_raw], axis=1)
potential_benchmarks_all.columns = ['potential_benchmarks_all']
df = pd.concat([df, potential_benchmarks_all], axis=1)

In [106]:
#get union of all sets
def groupby_occurences(sets):
    return collections.Counter([item for sublist in sets for item in sublist])


# cik_to_benchmarks = df.groupby('cik')['potential_benchmarks_all'].agg(lambda x: set.intersection(*x)).reset_index()
# cik_to_benchmarks = df.groupby('cik')['potential_benchmarks_all'].agg(lambda x: set.union(*x)).reset_index()
cik_to_benchmarks_occurrences = df.groupby('cik')['potential_benchmarks_all'].agg(lambda x: groupby_occurences(x)).reset_index()
cik_to_benchmarks_occurrences.columns = ['cik', 'potential_benchmark_occurrences']





In [107]:
def custom(dictionary):
    l = dictionary.keys()
    keys_to_keep = set([x for x in l if sum(1 for s in l if x in s) <= 1])
    new_dict = {k: dictionary[k] for k in keys_to_keep}

    tmp = {}
    tmp_sets = []
    for k, v in new_dict.items():
        splitted_k = set(k.split())
        if set(splitted_k) not in tmp_sets:
            tmp_sets.append(splitted_k)
            tmp[k] = v
        else:
            for i, j in tmp.items():
                if set(i.split()).issubset(splitted_k) or set(i.split()).issuperset(splitted_k):
                    tmp[i] += v
                    break

    return {k: v for k, v in sorted(tmp.items(), key=lambda item: item[1], reverse=True)}

df = pd.concat([cik_to_benchmarks_occurrences], axis=1)
cik_to_benchmarks_custom = df.potential_benchmark_occurrences.apply(lambda x: custom(x))

df2 = pd.concat([cik_to_benchmarks_custom], axis=1)
df2.columns=['benchmarks_custom']
# cik_to_benchmarks_custom_preprocessed = cik_to_benchmarks_occurrences_preprocessed.apply(lambda x:custom(x))
df = pd.concat([df, df2], axis=1)

In [108]:
df.columns

Index(['cik', 'potential_benchmark_occurrences', 'benchmarks_custom'], dtype='object')

In [109]:
# df = pd.concat([cik_to_benchmarks_occurrences, cik_to_benchmarks_occurrences_preprocessed], axis=1)

In [117]:
df.columns

Index(['cik', 'potential_benchmark_occurrences', 'benchmarks_custom'], dtype='object')

In [110]:
df.to_pickle('data/find_benchmark_1')

In [116]:
load_pkl('data/find_benchmark_1.pkl')[:20].to_markdown()

'|    |        cik | potential_benchmark_occurrences                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    